In [33]:
import random
from copy import deepcopy

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import interpolate
from scipy.stats import spearmanr
from sklearn.preprocessing import minmax_scale, robust_scale

In [34]:
data = pd.read_csv('data/1.data.csv', index_col=0)

In [35]:
data.head(25)

,DG 96910 - 3 MUSKETEERS FS 6PK 2.93OZ 24CA,DG 96910 - 3 MUSKETEERS MULTI-PIECE KINGSIZE 6/24CT,DG 96910 - 3 MUSKETEERS SINGLES 1.92OZ 36CT 10/CA,DG 96910 - COMBOS CHEDDAR PRETZEL SNGL 18CT 12/CA,DG 96910 - COMBOS PEPP PIZZA CRKR SNGL 18CT 12/CA,DG 96910 - COMBOS PIZZA PRETZEL SINGLES 18CT 12/CA,DG 96910 - DOVE MILK CHOC SINGLES 1.44OZ 18CT 12/CA,DG 96910 - M&MS MC MINIS MEGATUBE 1.77OZ 24CT 144CS,DG 96910 - M&MS MILK CHOC KINGSIZE 3.14OZ 24CT 6/CA,DG 96910 - M&MS PEANUT KING SIZE 3.27OZ 24CT 6/CA,...,DG DC 96970 - SKITTLES ORIGINAL SINGLES 36CT 10/CA,DG DC 96970 - SKITTLES SOURS SINGLES 1.8OZ 24CT 12/CA,DG DC 96970 - SKITTLES WILD BERRY SINGLES 36CT 10/CA,DG DC 96970 - STARBURST 14OZ LDBG 12/CA,DG DC 96970 - STARBURST FAVEREDS 14OZ LD BG 12/CA,DG DC 96970 - STARBURST ORIGINAL SINGLES 36CT 10/CA,DG DC 96970 - STB ORG JELBN BAG 14OZ 12/CS,DG DC 96970 - TWIX CARAMEL KS 3.02OZ 24CT 6/CA,DG DC 96970 - TWIX CARAMEL SNGL 1.79OZ 36CT 10/CA,DG DC 96970 - TWIX FS 6PK 3.28OZ 24CA
2016-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
tempDictionary = {}
for column in list(data):
    tempArray = list(data[column].values)
    tempArray = minmax_scale(tempArray)
    
    tempArray = tuple([str(value) for value in tempArray])
    
    if tempArray not in tempDictionary.keys():
        tempDictionary[tempArray] = [column]
    
    else:
        tempDictionary[tempArray].append(column)

for key, value in tempDictionary.items():
    
    if len(value) > 1:
        data.drop(value, axis = 1, inplace = True)
        
        value = [subValue.split('|') for subValue in value]
        print(len(value), value)
        
        value = [item for subValue in value for item in subValue]
        data['|'.join(sorted(list(set(value))))] = [float(value) for value in key]
        
        print(data.values.shape)

23 [['DG DC 6150 - 3 MUSKETEERS FS 6PK 2.93OZ 24CA'], ['DG DC 6150 - 3 MUSKETEERS MULTI-PIECE KINGSIZE 6/24CT'], ['DG DC 6150 - 3 MUSKETEERS SINGLES 1.92OZ 36CT 10/CA'], ['DG DC 6150 - COMBOS CHEDDAR PRETZEL SNGL 18CT 12/CA'], ['DG DC 6150 - COMBOS PEPP PIZZA CRKR SNGL 18CT 12/CA'], ['DG DC 6150 - COMBOS PIZZA PRETZEL SINGLES 18CT 12/CA'], ['DG DC 6150 - M&MS MC MINIS MEGATUBE 1.77OZ 24CT 144CS'], ['DG DC 6150 - M&MS MILK CHOC KINGSIZE 3.14OZ 24CT 6/CA'], ['DG DC 6150 - M&MS PEANUT KING SIZE 3.27OZ 24CT 6/CA'], ['DG DC 6150 - M&MS PEANUT SINGLES 1.74OZ 48CT 8/CA'], ['DG DC 6150 - MILKY WAY FS 6PK 3.36OZ 24CA'], ['DG DC 6150 - MILKY WAY MIDNIGHT SINGLE 1.76OZ 12/24CT'], ['DG DC 6150 - MILKY WAY SIMPLY CRML SNGL 1.91OZ 12/24C'], ['DG DC 6150 - MILKY WAY SINGLES 1.84OZ 36CT 10/CA'], ['DG DC 6150 - MILKYWAY SMP CARAMEL FS 6PK 4.42OZ 24CA'], ['DG DC 6150 - MMS MC $1.00 PEG PK 2.83OZ 12/CA'], ['DG DC 6150 - MMS PEANUT $1.00 PEG PK 2.83OZ 12/CA'], ['DG DC 6150 - SKITTLES ORIGINAL SINGLES 36CT

In [56]:
trajectoriesSet, trajectoriesSetMain, trajectoriesRaw = {}, {}, {}

for column in list(data):
    
    if column == 'index':
        continue
    
    tempDF = data[[column]]
    tempDF.fillna(0, inplace = True)
    
    if len(list(set([value for value in tempDF[column].values if np.isfinite(value)]))) == 1:
        continue
    
    tempDF.index = pd.to_datetime(tempDF.index, yearfirst = True)
    
    firstIndex = tempDF.first_valid_index()
    lastIndex = tempDF.last_valid_index()
    
    tempDFCut = tempDF.loc[firstIndex: lastIndex]
    tempArray = tempDFCut.resample('W').agg(np.nanmean)[column].values
    trajectoriesSet[column] = [list(robust_scale(tempArray))]
    
    tempArray = tempDF.resample('W').agg(np.nanmean)[column].values
    trajectoriesSetMain[column] = list(robust_scale(tempArray))
    
    trajectoriesRaw[column] = list(data[column].fillna(0))
    
    if len(trajectoriesSet.keys()) > 33:
        break

c:\__work\timeseriesclustering\venv\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\__work\timeseriesclustering\venv\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\__work\timeseriesclustering\venv\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\__work\timeseriesclustering\venv\lib\site-packages\pand

c:\__work\timeseriesclustering\venv\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\__work\timeseriesclustering\venv\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\__work\timeseriesclustering\venv\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\__work\timeseriesclustering\venv\lib\site-packages\pand

In [58]:
maxLength = max([len(value[0]) for _, value in trajectoriesSet.items()])

In [59]:
trajectoriesSetProcessed = {}
for key, value in trajectoriesSet.items():
    value = value[0]
    
    if len(value) == maxLength:
        trajectoriesSetProcessed[(key, )] = [value]
        continue
    
    oldScale = np.arange(0, maxLength, maxLength / len(value))
    
    oldScale = oldScale[: min([len(oldScale), len(value)])]
    value = value[: min([len(oldScale), len(value)])]
    
    try:
        interpolationFunction = interpolate.interp1d(oldScale, value)
    
    except:
        print(value)
    
    cutOff = 0
    while True:
        newScale = np.linspace(0, maxLength - cutOff, maxLength)
    
        try:
            value = interpolationFunction(newScale)
            break
            
        except:
            cutOff += 1
    
    trajectoriesSetProcessed[(key, )] = [value]

In [61]:
THRESHOLD = 0.50

trajectories = deepcopy(trajectoriesSetProcessed)
distanceMatrixDictionary = {}

iteration = 1
while True:
    distanceMatrix = np.empty((len(trajectories), len(trajectories),))
    distanceMatrix[:] = np.nan
    
    for index1, (filter1, trajectory1) in enumerate(trajectories.items()):
        tempArray = []
        
        for index2, (filter2, trajectory2) in enumerate(trajectories.items()):
            
            if index1 > index2:
                continue
            
            elif index1 == index2:
                continue
            
            else:
                unionFilter = filter1 + filter2
                sorted(unionFilter)
                
                if unionFilter in distanceMatrixDictionary.keys():
                    distanceMatrix[index1][index2] = distanceMatrixDictionary.get(unionFilter)
                    
                    continue
                
                metric = []
                for subItem1 in trajectory1:
                    length1 = len(subItem1)
                    
                    for subItem2 in trajectory2:                        
                        metric.append(-spearmanr(subItem1, subItem2)[0] + 1)
                
                metric = max(metric)
                
                distanceMatrix[index1][index2] = metric
                distanceMatrixDictionary[unionFilter] = metric
    
    minValue = np.min(list(distanceMatrixDictionary.values()))
    
    print(minValue)
    
    if minValue > THRESHOLD:
        print(minValue, THRESHOLD)
        break
    
    minIndices = np.where(distanceMatrix == minValue)
    minIndices = list(zip(minIndices[0], minIndices[1]))
    
    minIndex = minIndices[0]
    
    filter1 = list(trajectories.keys())[minIndex[0]]
    filter2 = list(trajectories.keys())[minIndex[1]]
    
    trajectory1 = trajectories.get(filter1)
    trajectory2 = trajectories.get(filter2)
    
    unionFilter = filter1 + filter2
    sorted(unionFilter)
    
    trajectoryGroup = trajectory1 + trajectory2
    
    trajectories = {key: value for key, value in trajectories.items()
                    if all(value not in unionFilter for value in key)}
    
    distanceMatrixDictionary = {key: value for key, value in distanceMatrixDictionary.items()
                                if all(value not in unionFilter for value in key)}
    
    trajectories[unionFilter] = trajectoryGroup
    
    print(iteration, 'finished!')
    iteration += 1
    
    if len(list(trajectories.keys())) == 1:
        break

0.3331448596157782
1 finished!
0.34976344451474806
2 finished!
0.4322961694745322
3 finished!
0.4470522841552844
4 finished!
0.44980184438798043
5 finished!
0.4526398870513705
6 finished!
0.4988029078254006
7 finished!
0.507256031910269
0.507256031910269 0.5


In [62]:
for key, value in trajectories.items():
    print(key, len(value))

('DG 96910 - 3 MUSKETEERS MULTI-PIECE KINGSIZE 6/24CT',) 1
('DG 96910 - COMBOS CHEDDAR PRETZEL SNGL 18CT 12/CA',) 1
('DG 96910 - COMBOS PEPP PIZZA CRKR SNGL 18CT 12/CA',) 1
('DG 96910 - M&MS MC MINIS MEGATUBE 1.77OZ 24CT 144CS',) 1
('DG 96910 - M&MS MILK CHOC KINGSIZE 3.14OZ 24CT 6/CA',) 1
('DG 96910 - M&MS PEANUT KING SIZE 3.27OZ 24CT 6/CA',) 1
('DG 96910 - M&MS PEANUT SINGLES 1.74OZ 48CT 8/CA',) 1
('DG 96910 - MILKY WAY FS 6PK 3.36OZ 24CA',) 1
('DG 96910 - MILKY WAY MIDNIGHT SINGLE 1.76OZ 12/24CT',) 1
('DG 96910 - MILKY WAY SIMPLY CRML SNGL 1.91OZ 12/24C',) 1
('DG 96910 - MILKY WAY SINGLES 1.84OZ 36CT 10/CA',) 1
('DG 96910 - MILKYWAY SMP CARAMEL FS 6PK 4.42OZ 24CA',) 1
('DG 96910 - SKITTLES ORIGINAL 14OZ 12CT',) 1
('DG 96910 - SKITTLES ORIGINAL SINGLES 36CT 10/CA',) 1
('DG 96910 - SKITTLES SOURS SINGLES 1.8OZ 24CT 12/CA',) 1
('DG 96910 - STARBURST 14OZ LDBG 12/CA',) 1
('DG 96910 - TWIX CARAMEL KS 3.02OZ 24CT 6/CA',) 1
('DG 96920 - 3 MUSKETEERS FS 6PK 2.93OZ 24CA',) 1
('DG 96920 - 3 M

In [63]:
clusterNames = sorted(list(set(trajectories.keys())))

'''nameColumn, idColumn = [], []
for clusterIndex, clusterName in enumerate(clusterNames):
    
    if len(clusterName) == 1:
        continue
    
    nameColumn.append('|'.join(clusterName))
    idColumn.append(clusterIndex + 1)

resultDF = pd.DataFrame()
resultDF['name'] = nameColumn
resultDF['id'] = idColumn

resultDF.to_csv('2.3.clusteringWeekly[correlation+interpolation].csv')'''

"nameColumn, idColumn = [], []\nfor clusterIndex, clusterName in enumerate(clusterNames):\n    \n    if len(clusterName) == 1:\n        continue\n    \n    nameColumn.append('|'.join(clusterName))\n    idColumn.append(clusterIndex + 1)\n\nresultDF = pd.DataFrame()\nresultDF['name'] = nameColumn\nresultDF['id'] = idColumn\n\nresultDF.to_csv('2.3.clusteringWeekly[correlation+interpolation].csv')"

In [64]:
for clusterIndex, clusterName in enumerate(clusterNames):
    
    if len(clusterName) == 1:
        continue
    
    figure = make_subplots(rows=3, cols=1)
    colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(clusterName))]
    
    value = []
    for subKey in clusterName:
        value.append(trajectoriesSet.get(subKey)[0])
    
    for index, subValue in enumerate(value):
        figure.add_trace(go.Scatter(x=list(range(0, len(subValue))), y=subValue,
                                    mode='lines', marker_color=colors[index], line = dict(width=2.5), line_shape='spline'), row=1, col=1)
    
    #
    
    value = []
    for subKey in clusterName:
        value.append(trajectoriesSetMain.get(subKey))
    
    for index, subValue in enumerate(value):
        figure.add_trace(go.Scatter(x=list(range(0, len(subValue))), y=subValue,
                                    mode='lines', marker_color=colors[index], line = dict(width=2.5), line_shape='spline'), row=2, col=1)
    
    value = []
    for subKey in clusterName:
        value.append(trajectoriesRaw.get(subKey))
        # value.append(trajectoriesSetProcessed.get((subKey, ))[0])
    
    for index, subValue in enumerate(value):
        figure.add_trace(go.Scatter(x=list(range(0, len(subValue))), y=subValue,
                                    mode='lines', marker_color=colors[index], line = dict(width=2.5), line_shape='spline'), row=3, col=1)
    
    #    
    
    figure.update_layout(title = 'Cluster ' + str(clusterIndex + 1), showlegend=False, height=1200, width=1200)
    figure.show()
    
    figure.write_image('images/cluster_' + str(clusterIndex + 1) + '.png')
    
    # break